# Natalia Organek - lab 5

In [204]:
import os
import requests
import regex as re
from collections import Counter
import nltk
import math
from tqdm import tqdm

In [3]:
directory = '../lab1/ustawy'
filenames = os.listdir(directory)

In [209]:
corpus = []
html_regex = r'<[\s\S]*?>'


for filename in filenames:
    with open(os.path.join(directory, filename), encoding="UTF-8") as f:
        bill = f.read()
        bill = bill.lower()
        bill = re.sub(html_regex, '', bill)
        bill = bill.replace('\xad', '').replace('\xa0', ' ')
        
        corpus.append(bill)

## Using KRNNT2 docker

In [116]:
def ask_krnnt(text):
    answer = requests.post('http://localhost:9201/', data=text.encode('utf-8')).text
    answer_words = answer.split('\n')
    words = [word for word in answer_words if word.startswith('\t')]
    return words

In [117]:
def lemmatize_and_tag_text(text):
    words = ask_krnnt(text)
    return [get_lem_tag(word) for word in words]
    
def get_lem_tag(word):
    splitted = word.split('\t')
    morf_cat = splitted[2].split(':')[0]
    return splitted[1] + ':' + morf_cat

In [172]:
lemmatized_corpus = lemmatize_and_tag_text('Ala chce kota, Ala ma kota i Ala ma kota, aczkolwiek przyszedłszy z dworu, nie nakarmiła go. \n\n\n ad.3 dz.u')
print(lemmatized_corpus)

['Ala:subst', 'chcieć:fin', 'kot:subst', ',:interp', 'Ala:subst', 'mieć:fin', 'kot:subst', 'i:conj', 'Ala:subst', 'mieć:fin', 'kot:subst', ',:interp', 'aczkolwiek:comp', 'przyjść:pant', 'z:prep', 'dwór:subst', ',:interp', 'nie:qub', 'nakarmić:praet', 'on:ppron3', '.:interp', 'ad:brev', '.:interp', '3:adj', 'dziennik:brev', '.:interp', 'u:prep']


In [214]:
lemmatized_corpus = []
for corpus_text in tqdm(corpus):
    lemmatized_corpus.extend(lemmatize_and_tag_text(corpus_text))

100%|██████████| 1179/1179 [1:00:34<00:00,  3.08s/it]


## Using the tagged corpus compute bigram statistic for the tokens containing:
 - lemmatized, downcased word
 - morphosyntactic category of the word (subst, fin, adj, etc.)

In [215]:
bigrams = [b for b in nltk.bigrams(lemmatized_corpus)]

In [249]:
bigrams[:10]

[('dziennik:brev', '.:interp'),
 ('.:interp', 'u:prep'),
 ('u:prep', '.:interp'),
 ('.:interp', 'z:prep'),
 ('z:prep', '1998:adj'),
 ('1998:adj', 'rok:brev'),
 ('rok:brev', '.:interp'),
 ('.:interp', 'numer:brev'),
 ('numer:brev', '75:num'),
 ('75:num', ',:interp')]

In [217]:
letters_reg = r'^\p{Letter}+$'

def token_valid(token):
    return re.match(letters_reg, token.split(':')[0])

In [218]:
bigrams_count = Counter(bigrams)

In [219]:
valid_bigrams = Counter({
    (t1, t2): count for (t1, t2), count in bigrams_count.items() if token_valid(t1) and token_valid(t2)
})

In [244]:
valid_bigrams.most_common()[:15]

[(('w:prep', 'artykuł:brev'), 32038),
 (('o:prep', 'który:adj'), 28658),
 (('który:adj', 'mowa:subst'), 28540),
 (('mowa:subst', 'w:prep'), 28473),
 (('w:prep', 'ustęp:brev'), 23536),
 (('z:prep', 'dzień:subst'), 11360),
 (('otrzymywać:fin', 'brzmienie:subst'), 10535),
 (('określić:ppas', 'w:prep'), 9693),
 (('do:prep', 'sprawa:subst'), 8718),
 (('ustawa:subst', 'z:prep'), 8625),
 (('właściwy:adj', 'do:prep'), 8536),
 (('i:conj', 'numer:brev'), 8435),
 (('dodawać:fin', 'się:qub'), 8196),
 (('minister:subst', 'właściwy:adj'), 7936),
 (('w:prep', 'brzmienie:subst'), 7278)]

In [240]:
valid_tokens = Counter([token for token in lemmatized_corpus if token_valid(token)])
tokens_len = sum(valid_tokens.values())

In [245]:
valid_tokens.most_common()[:15]

[('w:prep', 202691),
 ('i:conj', 90001),
 ('z:prep', 87985),
 ('artykuł:brev', 83766),
 ('o:prep', 64714),
 ('do:prep', 60758),
 ('ustęp:brev', 53338),
 ('na:prep', 50647),
 ('który:adj', 49385),
 ('się:qub', 45888),
 ('lub:conj', 45800),
 ('pozycja:brev', 45216),
 ('numer:brev', 44940),
 ('oraz:conj', 33564),
 ('rok:brev', 33137)]

## Compute LLR statistic for this dataset.

In [241]:
def denormEntropy(counts):
    '''Computes the entropy of a list of counts scaled by the sum of the counts. If the inputs sum to one, this is just the normal definition of entropy'''
    total = float(sum(counts))
    # Note tricky way to avoid 0*log(0)
    return -sum([k * math.log(k/total + (k==0)) for k in counts])

def llr_2x2(k11, k12, k21, k22):
    '''Special case of llr with a 2x2 table'''
    return 2 * (denormEntropy([k11+k12, k21+k22]) +
                denormEntropy([k11+k21, k12+k22]) -
                denormEntropy([k11, k12, k21, k22]))

def llr(word1, word2):
    k11 = valid_bigrams[(word1, word2)]
    k12 = valid_tokens[word2] - k11
    k21 = valid_tokens[word1] - k11
    k22 = tokens_len - k12 - k21 - k11
    return llr_2x2(k11, k12, k21, k22)


In [242]:
bigrams_llr = {(w1, w2): llr(w1, w2) for (w1, w2) in valid_bigrams.keys()}
bigrams_llr_list = [b for b in bigrams_llr.items()]
bigrams_llr_list.sort(key = lambda a: (-a[1], a[0][0]))

In [243]:
bigrams_llr_list[:10]

[(('który:adj', 'mowa:subst'), 262456.2132033333),
 (('o:prep', 'który:adj'), 178228.44435145345),
 (('mowa:subst', 'w:prep'), 163972.24869889254),
 (('otrzymywać:fin', 'brzmienie:subst'), 116130.94667432937),
 (('w:prep', 'artykuł:brev'), 82807.20136142336),
 (('minister:subst', 'właściwy:adj'), 71774.7355501341),
 (('dodawać:fin', 'się:qub'), 70893.8407511349),
 (('w:prep', 'ustęp:brev'), 67792.45630358532),
 (('stosować:fin', 'się:qub'), 56164.454882081365),
 (('droga:subst', 'rozporządzenie:subst'), 54199.295313896466)]

## Partition the entries based on the syntactic categories of the words,
i.e. all bigrams having the form of w1:adj w2:subst should be placed in one partition (the order of the words may not be changed).

In [246]:
def get_category(word):
    return word.split(':')[1]

def get_bigram_category(bigram):
    return get_category(bigram[0]), get_category(bigram[1])

In [247]:
partitions_dict = {}
for (bigram, llr) in bigrams_llr_list:
    category = get_bigram_category(bigram)
    if partitions_dict.get(category):
        partitions_dict[category].append((bigram, llr))
    else:
        partitions_dict[category] = [(bigram, llr)]

In [252]:
partitions_count = [(key, len(vals)) for key, vals in partitions_dict.items()]
partitions_count.sort(key = lambda a: (-a[1], a[0][0]))

In [253]:
partitions_count[:10]

[(('subst', 'subst'), 47929),
 (('subst', 'adj'), 27162),
 (('adj', 'subst'), 26170),
 (('subst', 'fin'), 16160),
 (('ger', 'subst'), 15587),
 (('prep', 'subst'), 12302),
 (('subst', 'prep'), 11375),
 (('subst', 'ppas'), 10708),
 (('fin', 'subst'), 8815),
 (('adj', 'fin'), 8701)]

In [254]:
top_10_partitions = {name: partitions_dict[name] for name, count in partitions_count[:10]}

## Use the computed LLR measure to select 5 bigrams for each of the largest categories.

In [256]:
for partition, words in top_10_partitions.items():
    print(partition)
    for word in words[:5]:
        print('\t', word)

('subst', 'subst')
	 (('droga:subst', 'rozporządzenie:subst'), 54199.295313896466)
	 (('skarb:subst', 'państwo:subst'), 22938.521533561434)
	 (('rada:subst', 'minister:subst'), 15832.100085449114)
	 (('terytorium:subst', 'rzeczpospolita:subst'), 14825.037587484869)
	 (('ochrona:subst', 'środowisko:subst'), 14682.174867568887)
('subst', 'adj')
	 (('minister:subst', 'właściwy:adj'), 71774.7355501341)
	 (('rzeczpospolita:subst', 'polski:adj'), 44555.91107030348)
	 (('jednostka:subst', 'organizacyjny:adj'), 25135.72813975817)
	 (('samorząd:subst', 'terytorialny:adj'), 24068.68876693108)
	 (('produkt:subst', 'leczniczy:adj'), 22191.420445811877)
('adj', 'subst')
	 (('który:adj', 'mowa:subst'), 262456.2132033333)
	 (('niniejszy:adj', 'ustawa:subst'), 21091.35283317353)
	 (('następujący:adj', 'zmiana:subst'), 18007.764785394815)
	 (('odrębny:adj', 'przepis:subst'), 12445.36233583247)
	 (('walny:adj', 'zgromadzenie:subst'), 9833.689830195068)
('subst', 'fin')
	 (('ustawa:subst', 'wchodzić:fin'

## Using the results from the previous step answer the following questions:
### What types of bigrams have been found?
Prawie wszystkie bigramy (wśród najpopularniejszych) zawierają rzeczownik, w większości wraz z jego określeniem w różnych klasach morfologicznych (np. rzeczownik z przymiotnikiem [symetrycznie przymiotnik z rzeczownikiem], rzeczownik z rzeczownikiem [największa klasa], przymiotnik z rzeczownikiem odczasownikowym, imiesłowem przym. biernym itp.) występują również połączenia rzeczownik-przyimek (symetrycznie przyimek-rzeczownik), rzeczownik-czasownik, czasownik rzeczownik.

Największ klasa bigramów które nie zawierają rzeczownika to przymiotnik-czasownik(non-past), jednak patrząc na przykłady tej klasy - są to części min. trigramów (... obowiązany jest, ... który wchodzi, ... wewnętrzny określa, itp, inaczej nie znamy pełnego podmiotu).


### Which of the category-pairs indicate valuable multiword expressions? Do they have anything in common?
Są to wyrażenia, które mają przynajmniej jeden rzeczownik. Pary rzeczownik-rzeczownik to są praktycznie tylko prawidłowe wyrażenia wielowyrazowe (oczywiście patrząc na te przykłady z top-LLR), np. skarb państwa, ohrona środowiska.

Ciekawa rzecz jest przy bigramach rzeczownik-przymniotnik, gdyż w tej kolejności również są niemal tylko MWE, (minister właściwy, produkt leczniczy), ale gdy kolejność zostaje odwócona (przymiotnik-rzeczownik), to pojawiają się już bigramy, które ciężko zakwalifikować jako MWE (który - mowa), oczywiście istnieją i takie (walne zgromadzenie) i jest ich też dość dużo.

Jeśli chodzi o pary rzeczownik-czasownik w formie prostej(fin) i czasownik-rzeczownik, to trudno je kwalifikować jako MWE, w większości są to pary słów często stojące obok siebie, ale niemające dodatkowego/innego znaczenia. Podobnie jest w przypadku wyrażeń z przyimkami. Podobnie ma się rzecz z parami rzeczownik-rzeczownik odczasownikowy czy rzeczownik-imiesłów przymiotnikowy bierny.

Przyimki z rzeczownikami nie tworzą MWE, tak samo jak przymiotnik z czasownikiem(fin) - w tym wypadku, tak jak pisałam wcześniej, są to głównie części trigramów, więc ciężko oszukiwać się w nich MWE.


### Which signal: LLR score or syntactic category is more useful for determining genuine multiword expressions?
Myślę że ich połączenie działa dosyć dobrze, tzn. najpierw wyodrębnienie kategorii morfologicznych, a dopiero w ich obrębie sortowanie po wyniku LLR. Wyodrębnienie kategorii jest kluczowe, gdyż widać, że niektóre ich połączenia mają tendencję do tworzenia związków frazeologicznych, a niektóre po prostu nie. W poprzednim laboratorium wyniki po samym LLR były mierne - po prostu były to częste połączenia wyrazowe, ale mało miały wspólnego z MWE. W dodatku nie zastosowaliśmy wtedy lematyzacji, więc były do siebie bardzo podobne.



### Can you describe a different use-case where the morphosyntactic category is useful for resolving a real-world problem?

Wyszukiwanie nazw własnych w tekście; sprawdzanie poprawności budowanych zdań, albo nawet ich rozbudowywanie (podpowiedzi od aplikacji, jakie słowa/wyrażenia mogłyby pasować, wcześniej oczywiście system musiałby wiedzieć jakie kategorie mogą stać po jakich, potem jakie są częste połączenia konkretnych wyrazów).